In [1]:
def selectk_best(indep,dep,n):
    from sklearn.feature_selection import SelectKBest,chi2
    selector=SelectKBest(score_func=chi2,k=n)
    fit1=selector.fit(indep,dep)
    x=fit1.transform(indep)
    feature=fit1.get_support(indices=True)
    col=data.columns[feature]
    return x,col

def split_scale(x,dep):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    x_train,x_test,y_train,y_test=train_test_split(x,dep,test_size=1/3,random_state=0)
    sc=StandardScaler()
    x_train_sc=sc.fit_transform(x_train)
    x_test_sc=sc.transform(x_test)
    return x_train_sc,x_test_sc,y_train,y_test 

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

def logistic(x_train_sc,x_test_sc,y_train,y_test):
    from sklearn.linear_model import LogisticRegression
    model=LogisticRegression()
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    accuracy=accuracy_score(y_pred,y_test)
    return accuracy

def svml(x_train_sc,x_test_sc,y_train,y_test):
    from sklearn.svm import SVC
    model=SVC(kernel='linear')
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    accuracy=accuracy_score(y_pred,y_test)
    return accuracy


def svm_nl(x_train_sc,x_test_sc,y_train,y_test):
    from sklearn.svm import SVC
    model=SVC(kernel='rbf')
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    accuracy=accuracy_score(y_pred,y_test)
    return accuracy

def naive(x_train_sc,x_test_sc,y_train,y_test):
    from sklearn.naive_bayes import GaussianNB
    model=GaussianNB()
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    accuracy=accuracy_score(y_pred,y_test)
    return accuracy

def knn(x_train_sc,x_test_sc,y_train,y_test):
    from sklearn.neighbors import KNeighborsClassifier
    model=KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    accuracy=accuracy_score(y_pred,y_test)
    return accuracy

def Decision(x_train_sc,x_test_sc,y_train,y_test):
    from sklearn.tree import DecisionTreeClassifier
    model= DecisionTreeClassifier(criterion='entropy')
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    accuracy=accuracy_score(y_pred,y_test)
    return accuracy

def random_forest(x_train_sc,x_test_sc,y_train,y_test):
    from sklearn.ensemble import RandomForestClassifier
    model= RandomForestClassifier(n_estimators=10,criterion='entropy')
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    accuracy=accuracy_score(y_pred,y_test)
    return accuracy


In [2]:
import pandas as pd
data=pd.read_csv("CKD.csv")

In [3]:
data=pd.get_dummies(data,drop_first=True)

In [4]:
indep=data.drop('classification_yes',axis=1)
dep=data['classification_yes']

In [5]:
def combo(indep,dep,n):
    kbest,selected_columns=selectk_best(indep,dep,n)
    x_train_sc,x_test_sc,y_train,y_test=split_scale(kbest,dep)
    log_accuracy=logistic(x_train_sc,x_test_sc,y_train,y_test)
    svm_accuracy=svml(x_train_sc,x_test_sc,y_train,y_test)
    svm_nl_accuracy=svm_nl(x_train_sc,x_test_sc,y_train,y_test)
    naive_accuracy=naive(x_train_sc,x_test_sc,y_train,y_test)
    knn_accuracy=knn(x_train_sc,x_test_sc,y_train,y_test)
    decision_accuracy=Decision(x_train_sc,x_test_sc,y_train,y_test)
    random_accuracy=random_forest(x_train_sc,x_test_sc,y_train,y_test)
    chiTable=pd.DataFrame(index=['Chi-Square'],columns=['Logistic','SVML','SVMnL','KNN','Naive','Decision','Random'])
    chiTable.loc['Chi-Square','Logistic']=log_accuracy
    chiTable.loc['Chi-Square','SVML']=svm_accuracy
    chiTable.loc['Chi-Square','SVMnL']=svm_nl_accuracy
    chiTable.loc['Chi-Square','KNN']=knn_accuracy
    chiTable.loc['Chi-Square','Naive']=naive_accuracy
    chiTable.loc['Chi-Square','Decision']=decision_accuracy
    chiTable.loc['Chi-Square','Random']=random_accuracy
    #return chiTable,selected_columns
    print(f"Table: \n{chiTable}")
    
    print(f"K best Columns: {selected_columns}")







In [6]:
combo(indep,dep,5)

Table: 
            Logistic      SVML     SVMnL       KNN     Naive  Decision  \
Chi-Square  0.947368  0.947368  0.954887  0.917293  0.857143  0.962406   

              Random  
Chi-Square  0.954887  
K best Columns: Index(['bgr', 'bu', 'sc', 'pcv', 'wc'], dtype='object')
